## STEP 01. Import Modules

In [1]:
import numpy as np
import pandas as pd
import json
from tqdm import *

import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt

c:\python38\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
import os
os.getcwd();

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## STEP 02. Set Target csv File List

In [1]:
list = [ 'news']

## STEP 03. PREPROCESSING - OPTION

In [3]:
# For user analysis, sometimes spamming and repetitve crawling results are what you need.
# So when you need those results, you can skip this STEP or do it in part.

for name in tqdm(list) :
    name_01 = name + ".csv"    
    df = pd.read_csv(name_01).dropna()
    df.reset_index(drop=True, inplace=True)
       
    print("삭제 대상을 확인 중입니다.")
    
    for i in tqdm(range(len(df))) :
        try :
            target_01 = len( [ x for x in df['title'] if x == df['title'][i] ])
        except :
            target_01 = len( [ x for x in df['user_text'] if x == df['user_text'][i] ])
            
        target_02 = len( [ y for y in df['url'] if y == df['url'][i] ])
        
        if target_01 > 1 :
            df['title'][i] = "delete target"
        elif target_02 > 1 :
            df['url'][i] = "delete target"
    
    target_list = []
    
    print("삭제 대상을 제거합니다.")
    
    before = len(df)
    
    for j in range(len(df)) :
        if df['title'][j] == "delete target" or df['url'][j] == "delete target" :
            target_list.append(j)
            
    df.drop(df.index[target_list], inplace=True)
    
    after = len(df)
    
    print(f"file name {name} 작업 완료...")
    print(f"총 {before - after}개의 열이 삭제되었습니다.")
    
    if os.path.isdir("./preprocessing") :
        pass
    else :
        os.mkdir("./preprocessing")
    
    df.to_csv(f"./preprocessing/{name_01}", index=False)
    print(f"파일 저장 완료...저장 위치 : ./preprocessing/{name_01}")
    

print("preprocessing 작업을 완료하였습니다.")

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


삭제 대상을 확인 중입니다.


KeyError: 'user_text'

## STEP 04. Drawing LDA

In [11]:
df = pd.read_csv("news.csv")
df.columns = ['domain', 'keyword', 'user_text', 'channel', 'date']
df.to_csv("news.csv", index=False, encoding='utf-8-sig')

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
df = pd.read_csv("news.csv")
df

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,domain,keyword,user_text,channel,date
0,DaumNews,4주년연설,[나이트포커스] 박준영 자진 사퇴..청문 정국 전망은?,YTN,11시간전
1,DaumNews,이낙연,대선 양자대결서 이재명 42% 윤석열 35.1%..첫 역전(종합),뉴시스,11시간전
2,DaumNews,이낙연,"이재명, 가상 양자대결에서 처음으로 윤석열 앞질러",YTN,11시간전
3,DaumNews,이낙연,[나이트포커스] 이재명-윤석열 양강 구도 굳어지나?,YTN,11시간전
4,DaumNews,이낙연,"이낙연, “청년 안심사회, 청년 사회안전망 만들겠다”",더코리아,11시간전
...,...,...,...,...,...
16642,NaverNews,이재명,"""기본소득 공동체 신뢰 회복 계기 될 것""",강원도민일보,7일 전
16643,NaverNews,이재명,"“지지율 10% 목표”… 정세균, 이재명과 각 세워 ‘양강’ 노린다",국민일보언론사 선정,7일 전
16644,NaverNews,이재명,돈으로 청년 표심 사려는 대권주자들 행태는 청년 모욕이다 [사설],매일경제언론사 선정,7일 전
16645,NaverNews,이재명,김두관이 꺼낸 ‘대선 경선연기론’…여권 1위 이재명 측 반발,중앙일보,7일 전


In [2]:
list = ['comu']

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
for name in list :
    name_01 = name + ".csv"
    
    df = pd.read_csv(name_01).dropna()
    
    # Convert to list
    data = df.user_text.values.tolist()

    # user_text에서 명사만 추출 

    from konlpy.tag import *
    from nltk import Text

    okt = Okt()

    data_morph = []
    for i in range(len(data)):

        try :
            data_morph.append(" ".join(okt.nouns(data[i])))
        except : # 중간에 비어있는 글자, na null 등 건너뛰기
#             print("scout 00 :", data[i])
            pass


#     vectorizer = HashingVectorizer(analyzer='word',       
#                              n_features=50000,             
#                             )
    vectorizer = CountVectorizer(analyzer='word',       
                             min_df=3000,   # 특정 횟수 이상 언급된 것만 따로                      
                             max_features=50000,             
                            )

    data_vectorized = vectorizer.fit_transform(data_morph)

#     break

    NUM_TOPICS = 3 # 분류하는 카테고리 숫자
    
    lda_model = LatentDirichletAllocation(
                            n_components=NUM_TOPICS,              
                            max_iter=10,               
                            learning_method='online',   
                            random_state=100,          
                            batch_size=5000,        
                            evaluate_every = -1,       
                            #n_jobs = -1, 
        
                        )
    lda_output = lda_model.fit_transform(data_vectorized)
    log = lda_model.score(data_vectorized)
    per = lda_model.perplexity(data_vectorized)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    print(df_document_topics)
    
    name_02 = name + "_reference" + ".html"
        
    pyLDAvis.enable_notebook()
    p = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
    
    pyLDAvis.save_html(p,name_02)
    
        # Show top n keywords for each topic
    def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
        keywords = np.array(vectorizer.get_feature_names())
        topic_keywords = []
        for topic_weights in lda_model.components_:
            top_keyword_locs = (-topic_weights).argsort()[:n_words]
            topic_keywords.append(keywords.take(top_keyword_locs))
        return topic_keywords

    topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=11) 
    
    name_03 = name + "_reference" +".csv"
    pd.DataFrame(topic_keywords).to_csv(name_03, index=False, encoding='utf-8-sig')
    print(pd.DataFrame(topic_keywords));

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


   0   1    2   3   4    5    6   7   8   9   10
0  우리  진짜   이제  새끼  한번   시간   다시  오늘  요즘  저런  친구
1  그냥  정도   문제  정말  보고   때문  기레기  추천  백신  다른  미국
2  사람  생각  민주당  지금  하나  대통령   언론  댓글  의원  개혁  소리


In [ ]:
data_vectorized.to_array()

In [27]:
data_vectorized.shape

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(100, 50000)

In [29]:
print(data_vectorized[0])

  (0, 14749)	-0.5
  (0, 38577)	0.5
  (0, 42179)	0.5
  (0, 42519)	-0.5


c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
scaler = MinMaxScaler()
scaler.fit_transform(data_vectorized)

In [15]:
print(data_vectorized)

  (0, 11)	1
  (1, 24)	1
  (4, 22)	1
  (7, 22)	1
  (7, 7)	2
  (7, 6)	1
  (7, 15)	1
  (7, 18)	1
  (11, 35)	1
  (11, 28)	1
  (12, 43)	1
  (13, 35)	2
  (14, 32)	1
  (14, 34)	1
  (15, 15)	1
  (15, 32)	1
  (16, 32)	1
  (17, 0)	1
  (18, 35)	1
  (18, 0)	1
  (19, 35)	1
  (19, 32)	1
  (22, 43)	1
  (22, 37)	2
  (23, 32)	1
  :	:
  (91, 19)	1
  (92, 24)	1
  (92, 31)	1
  (92, 5)	2
  (92, 29)	1
  (92, 26)	1
  (92, 19)	1
  (93, 24)	1
  (93, 21)	1
  (93, 5)	1
  (93, 40)	1
  (93, 19)	1
  (94, 31)	1
  (94, 5)	1
  (94, 29)	1
  (94, 26)	1
  (95, 34)	1
  (95, 21)	1
  (95, 5)	1
  (95, 40)	1
  (95, 19)	3
  (96, 0)	1
  (98, 4)	1
  (98, 1)	1
  (99, 1)	1


c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
###### pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.121770 -0.350418       1        1  39.274023
1     -0.260912  0.274659       2        1  33.297876
0      0.382682  0.075759       3        1  27.428101, topic_info=    Term          Freq         Total Category  logprob  loglift
110   사람  64383.000000  64383.000000  Default  30.0000  30.0000
119   생각  54942.000000  54942.000000  Default  29.0000  29.0000
169   우리  26712.000000  26712.000000  Default  28.0000  28.0000
241   진짜  26480.000000  26480.000000  Default  27.0000  27.0000
34    그냥  29812.000000  29812.000000  Default  26.0000  26.0000
..   ...           ...           ...      ...      ...      ...
10    거기   7560.897346   7562.384944   Topic3  -4.3074   1.2934
4     가장   6788.611655   6792.619374   Topic3  -4.4151   1.2930
228   제일   5896.937816   5897.791340   Topic3  -4.5559   1.2935
134   시작   8858.334494   9024.680736   Topic3  -4.1490   1.2750
50    누구   6331.473469   6354.947978   Topic3  -4.4848   1.2899

[144 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         2  0.999847   가격
3         2  0.999854   가요
4         1  0.000442   가장
4         2  0.000147   가장
4         3  0.999467   가장
...     ...       ...  ...
261       2  0.999888   하루
262       2  0.999932   한국
263       3  0.999924   한번
274       2  0.999899   회사
276       1  0.999955   후보

[159 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

In [8]:
for name in list :
    name_01 = name + ".csv"
    keyword = "이재명"
    
    df = pd.read_csv(name_01).dropna()
    df2 = df[df['keyword'] == keyword]
    # Convert to list
    data = df2.user_text.values.tolist()

    # user text에서 명사만 추출 

    from konlpy.tag import *
    from nltk import Text

    okt = Okt()

    data_morph = []
    for i in range(len(data)):
        data_morph.append(" ".join(okt.nouns(data[i])))

    vectorizer = CountVectorizer(analyzer='word',       
                             min_df=5,   # 특정 횟수 이상 언급된 것만 따로                      
                             max_features=50000,             
                            )

    data_vectorized = vectorizer.fit_transform(data_morph)


    NUM_TOPICS = 3 # 분류하는 카테고리 숫자
    lda_model = LatentDirichletAllocation(
                            n_components=NUM_TOPICS,              
                            max_iter=10,               
                            learning_method='online',   
                            random_state=100,          
                            batch_size=128,            
                            evaluate_every = -1,       
                            #n_jobs = -1,               
                        )
    lda_output = lda_model.fit_transform(data_vectorized)
    log = lda_model.score(data_vectorized)
    per = lda_model.perplexity(data_vectorized)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    print(df_document_topics)
    
    name_02 = keyword + "_reference" + ".html"
        
    pyLDAvis.enable_notebook()
    p = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
    
    pyLDAvis.save_html(p,name_02)
    
        # Show top n keywords for each topic
    def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
        keywords = np.array(vectorizer.get_feature_names())
        topic_keywords = []
        for topic_weights in lda_model.components_:
            top_keyword_locs = (-topic_weights).argsort()[:n_words]
            topic_keywords.append(keywords.take(top_keyword_locs))
        return topic_keywords

    topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=11) 
    
    name_03 = keyword +  "_reference" +".csv"
    pd.DataFrame(topic_keywords).to_csv(name_03, index=False, encoding='utf-8-sig')
    print(pd.DataFrame(topic_keywords));

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


    0    1   2    3   4   5    6   7    8    9   10
0  대통령  지지자  이제  문재인  지금  때문   국민  개혁   검찰  이재명  정책
1  이재명  민주당  대선   지사  후보  지지  이낙연  경선  대통령   생각  의원
2   사람   생각  그냥   문제  진짜  하나   공격  정도   보고   자기  무슨


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.056351 -0.214879       1        1  45.180909
2     -0.232636  0.063023       2        1  27.493746
0      0.176285  0.151856       3        1  27.325345, topic_info=    Term        Freq       Total Category  logprob  loglift
662  이재명  925.000000  925.000000  Default  30.0000  30.0000
406   사람  308.000000  308.000000  Default  29.0000  29.0000
332  민주당  296.000000  296.000000  Default  28.0000  28.0000
802   지사  163.000000  163.000000  Default  27.0000  27.0000
124   그냥   90.000000   90.000000  Default  26.0000  26.0000
..   ...         ...         ...      ...      ...      ...
796   지금   54.819802  111.151470   Topic3  -4.4699   0.5905
549   언론   43.523141   85.355341   Topic3  -4.7007   0.6238
708   장관   23.436677   33.335958   Topic3  -5.3197   0.9450
662  이재명   48.967617  925.722163   Topic3  -4.5828  -1.6421
233   대해   23.656279   46.806867   Topic3  -5.3103   0.6150

[186 rows x 6 columns], token_table=      Topic      Freq Term
term                      
9         1  0.326431   가장
9         2  0.652862   가장
9         3  0.027203   가장
12        3  0.961032   가짜
16        2  0.962747   갈라
...     ...       ...  ...
978       2  0.059914   회사
978       3  0.958618   회사
980       1  0.741777   후보
980       2  0.129811   후보
980       3  0.129811   후보

[249 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

In [7]:
for name in list :
    name_01 = name + ".csv"
    keyword = "이낙연"
    
    df = pd.read_csv(name_01).dropna()
    df2 = df[df['keyword'] == keyword]
    # Convert to list
    data = df2.user_text.values.tolist()

    # user text에서 명사만 추출 

    from konlpy.tag import *
    from nltk import Text

    okt = Okt()

    data_morph = []
    for i in range(len(data)):
        data_morph.append(" ".join(okt.nouns(data[i])))

    vectorizer = CountVectorizer(analyzer='word',       
                             min_df=5,   # 특정 횟수 이상 언급된 것만 따로                      
                             max_features=50000,             
                            )

    data_vectorized = vectorizer.fit_transform(data_morph)


    NUM_TOPICS = 3 # 분류하는 카테고리 숫자
    lda_model = LatentDirichletAllocation(
                            n_components=NUM_TOPICS,              
                            max_iter=10,               
                            learning_method='online',   
                            random_state=100,          
                            batch_size=128,            
                            evaluate_every = -1,       
                            #n_jobs = -1,               
                        )
    lda_output = lda_model.fit_transform(data_vectorized)
    log = lda_model.score(data_vectorized)
    per = lda_model.perplexity(data_vectorized)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    print(df_document_topics)
    
    name_02 = keyword + "_reference" + ".html"
        
    pyLDAvis.enable_notebook()
    p = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
    
    pyLDAvis.save_html(p,name_02)
    
        # Show top n keywords for each topic
    def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
        keywords = np.array(vectorizer.get_feature_names())
        topic_keywords = []
        for topic_weights in lda_model.components_:
            top_keyword_locs = (-topic_weights).argsort()[:n_words]
            topic_keywords.append(keywords.take(top_keyword_locs))
        return topic_keywords

    topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=11) 
    
    name_03 = keyword +  "_reference" +".csv"
    pd.DataFrame(topic_keywords).to_csv(name_03, index=False, encoding='utf-8-sig')
    print(pd.DataFrame(topic_keywords));

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


    0    1   2    3     4   5   6   7    8    9   10
0   사람   청년  그냥  대통령    정치  문제  의원  주거   이제  민주당  정책
1   생각   진짜  본인   반성  최저임금  교대  우리  이상   사회   책임  선거
2  이낙연  민주당  사면   대표   이재명  경선  발언  대선  지지자   시간  후보


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.116471  0.209495       1        1  44.254671
0     -0.140438 -0.196859       2        1  29.558794
1      0.256909 -0.012636       3        1  26.186535, topic_info=    Term        Freq       Total Category  logprob  loglift
144   사람   91.000000   91.000000  Default  30.0000  30.0000
145   사면   93.000000   93.000000  Default  29.0000  29.0000
87    대표   85.000000   85.000000  Default  28.0000  28.0000
221  이낙연  127.000000  127.000000  Default  27.0000  27.0000
298   청년   41.000000   41.000000  Default  26.0000  26.0000
..   ...         ...         ...      ...      ...      ...
26    계속   13.024112   23.899657   Topic3  -4.4885   0.7329
61    남자   11.579691   17.748771   Topic3  -4.6061   0.9129
266   조국   11.239567   25.384449   Topic3  -4.6359   0.5252
95    때문   10.849400   21.433977   Topic3  -4.6712   0.6591
47    그냥   11.985421   50.563068   Topic3  -4.5716  -0.0996

[168 rows x 6 columns], token_table=      Topic      Freq Term
term                      
1         2  0.946422   가구
1         3  0.067602   가구
6         2  0.101046   가장
6         3  0.909413   가장
7         2  0.945485   가지
...     ...       ...  ...
317       3  0.177685   페미
321       1  0.910776   포럼
337       2  0.931742  화이팅
339       3  0.907350   회사
340       1  0.988242   후보

[203 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])

In [6]:
for name in list :
    name_01 = name + ".csv"
    keyword = "4주년연설"
    
    df = pd.read_csv(name_01).dropna()
    df2 = df[df['keyword'] == keyword]
    # Convert to list
    data = df2.user_text.values.tolist()

    # user text에서 명사만 추출 

    from konlpy.tag import *
    from nltk import Text

    okt = Okt()

    data_morph = []
    for i in range(len(data)):
        data_morph.append(" ".join(okt.nouns(data[i])))

    vectorizer = CountVectorizer(analyzer='word',       
                             min_df=5,   # 특정 횟수 이상 언급된 것만 따로                      
                             max_features=50000,             
                            )

    data_vectorized = vectorizer.fit_transform(data_morph)


    NUM_TOPICS = 3 # 분류하는 카테고리 숫자
    lda_model = LatentDirichletAllocation(
                            n_components=NUM_TOPICS,              
                            max_iter=10,               
                            learning_method='online',   
                            random_state=100,          
                            batch_size=128,            
                            evaluate_every = -1,       
                            #n_jobs = -1,               
                        )
    lda_output = lda_model.fit_transform(data_vectorized)
    log = lda_model.score(data_vectorized)
    per = lda_model.perplexity(data_vectorized)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    print(df_document_topics)
    
    name_02 = keyword + "_reference" + ".html"
        
    pyLDAvis.enable_notebook()
    p = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
    
    pyLDAvis.save_html(p,name_02)
    
        # Show top n keywords for each topic
    def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
        keywords = np.array(vectorizer.get_feature_names())
        topic_keywords = []
        for topic_weights in lda_model.components_:
            top_keyword_locs = (-topic_weights).argsort()[:n_words]
            topic_keywords.append(keywords.take(top_keyword_locs))
        return topic_keywords

    topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=11) 
    
    name_03 = keyword +  "_reference" +".csv"
    pd.DataFrame(topic_keywords).to_csv(name_03, index=False, encoding='utf-8-sig')
    print(pd.DataFrame(topic_keywords));

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


   0    1   2   3    4    5   6   7   8   9   10
0  그냥  대통령  진짜  보고   댓글   백신  기자  한번  생각  사진  오늘
1  사람   생각  페미  문제  기레기   하나  도장  지금  소리  개혁  때문
2  우리   국민  경제  생각   정말  코로나  정부  정도  위기  진입  접종


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.069933 -0.265888       1        1  40.638191
1     -0.275639  0.074999       2        1  35.248515
0      0.205706  0.190889       3        1  24.113294, topic_info=     Term        Freq       Total Category  logprob  loglift
753    사람  403.000000  403.000000  Default  30.0000  30.0000
209    그냥  201.000000  201.000000  Default  29.0000  29.0000
388   대통령  220.000000  220.000000  Default  28.0000  28.0000
1535   진짜  139.000000  139.000000  Default  27.0000  27.0000
1688   페미  167.000000  167.000000  Default  26.0000  26.0000
...   ...         ...         ...      ...      ...      ...
1001   얘기   51.327884   59.058567   Topic3  -5.0427   1.2821
508    말씀   50.905866   80.282133   Topic3  -5.0509   0.9668
802    생각   79.135339  386.779010   Topic3  -4.6097  -0.1643
1380   정도   52.270523  150.548464   Topic3  -5.0245   0.3646
798    상황   43.861299   99.313409   Topic3  -5.1999   0.6052

[164 rows x 6 columns], token_table=      Topic      Freq Term
term                      
2         3  0.982361   가격
17        2  0.991133   가요
24        2  0.993583   가지
63        1  0.085048   개혁
63        2  0.903632   개혁
...     ...       ...  ...
1748      1  0.011974   한번
1748      2  0.023947   한번
1748      3  0.957884   한번
1822      1  0.986599   회복
1823      3  0.994292   회사

[190 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])